# <font color='blue'>Séries Temporais</font>
<p> Séries Temporais nada mais é do que uma série de ponto de dados ordenados no decorrer do tempo. Em uma série temporal, o tempo costuma ser a variável independente e a meta geralmente é fazer uma previsão para o futuro.<br>

### Exemplos de campos de uso
<p>
Economia: Produto Interno Bruto (PIB), Índice de Preços ao Consumidor (IPC), Índice S&P 500 e taxas de desemprego<br>
Ciências Sociais: Taxas de natalidade, população, dados de migração, indicadores políticos<br>
Epidemiologia: Taxas de doenças, taxas de mortalidade, populações de mosquitos<br>
Medicina:Controle de pressão arterial, controle de peso, medições de colesterol, monitoramento de frequência cardíaca<br>
Física:	Temperaturas globais, observações mensais de manchas solares, níveis de poluição.    
    
# <font color='blue'>Monte Carlo</font>
<p> As simulações de Monte Carlo têm o nome do hot spot de apostas em Mônaco, pois o 
acaso e os resultados aleatórios são centrais na técnica de modelagem, assim como em jogos 
como roleta, dados e caça-níqueis. A técnica foi desenvolvida pela primeira vez por Stanislaw 
Ulam, um matemático que trabalhou no Projeto Manhattan. Após a guerra, enquanto se 
recuperava de uma cirurgia no cérebro, Ulam se divertiu jogando inúmeros jogos de paciência. 
Ele ficou interessado em traçar o resultado de cada um desses jogos, a fim de observar sua 
distribuição e determinar a probabilidade de vitória. Depois que ele compartilhou sua ideia com 
John Von Neumann, os dois colaboraram para desenvolver a simulação de Monte Carlo
    
### Exemplos de campos de uso
As telecomunicações as utilizam para avaliar o desempenho da rede em 
diferentes cenários, ajudando-os a otimizar a rede.<br> 
Os analistas os utilizam para avaliar o risco de uma entidade adiar e analisar derivativos, ou outros produtos financeiros.<br> Seguradoras e perfuradores de poços de petróleo também os utilizam.<br> 
As simulações de Monte Carlo têm inúmeras aplicações fora dos negócios e das finanças, como meteorologia, astronomia e física 
de partículas    

## Estudo de Caso - Simulação de Monte Carlo e Séries Temporais Para Modelagem Financeira

In [ ]:
# !pip3 install yfinance

In [7]:
# Import da bibli do YahooFiance para baixar cotações
import yfinance as yF

# Imports para manipulação de dados
import numpy as np
import pandas as pd

# import para visualizações
import matplotlib.pyplot as plt
import matplotlib as m
import seaborn as sns

# import para cálculos estatísticos
import scipy
from scipy.stats import kurtosis, skew, shapiro
import warnings
warnings.filterwarnings('ignore')

# import para formatação dos gráficos
plt.style.use('fivethirtyeight')
m.rcParams['axes.labelsize'] = 14
m.rcParams['xtick.labelsize'] = 12
m.rcParams['ytick.labelsize'] = 12
m.rcParams['text.color'] = 'k'
from matplotlib.pylab import rcParams
m.rcParams['figure.figsize'] = 20, 10

# formatando valores com duas casas decimais
pd.options.display.float_format = '{:.2f}'.format


## Carregando os Dados

In [9]:
# Carrega o dataset
#dados = pd.read_csv("dados.csv", parse_dates = True, index_col = "Data")

cotacoes = yF.Ticker('BTC-USD')

# Opções de períodos 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y e ytd.
dados = cotacoes.history(period='1y')
dados.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-01-12 00:00:00+00:00,18117.59,19030.09,17995.20,18869.59,34971338710,0.00,0.00
2023-01-13 00:00:00+00:00,18868.91,19964.32,18753.16,19909.57,29225029694,0.00,0.00
2023-01-14 00:00:00+00:00,19910.54,21075.14,19907.83,20976.30,38967784639,0.00,0.00
2023-01-15 00:00:00+00:00,20977.48,20993.75,20606.99,20880.80,19298407543,0.00,0.00
2023-01-16 00:00:00+00:00,20882.22,21360.88,20715.75,21169.63,26792494050,0.00,0.00
